In [1]:
import sys
sys.path.insert(0, '/home/sirine/PHD-work/snntorch-nir/snntorch')
import snntorch as snn
import torch
import nir
from snntorch import export_to_nir, import_from_nir

In [2]:
# 2D sample data 
sample_data = torch.randn(1, 1, 28, 28)  # input image size is 28x28 and 1 channel

class NetWithAvgPool(torch.nn.Module):
    def __init__(self):
        super(NetWithAvgPool, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.lif1 = snn.Leaky(beta=0.9, init_hidden=True)
        self.fc1 = torch.nn.Linear(28*28*16 // 4, 500)  # Adjusting input size after pooling layer
        self.lif2 = snn.Leaky(beta=0.9, init_hidden=True, output=True)

    def forward(self, x):
        x = torch.nn.functional.avg_pool2d(self.conv1(x), kernel_size=2, stride=2)  #  kernel_size=2 and stride=2 for avg_pool2d
        x = x.view(-1, 28*28*16 // 4)  # Adjusting the view based on the output shape after pooling
        x = self.lif1(x)
        x = self.fc1(x)
        x = self.lif2(x)
        return x

net_with_avgpool = NetWithAvgPool()
nir_graphtest = export_to_nir(net_with_avgpool, sample_data, ignore_dims=[0])

In [3]:
nir_graphtest.infer_types()

[warning] conv1.input_type undefined, set to input.output_type
[warning] overwriting output.input: [[500]] with conv1.output: [[16 28 28]]


In [4]:
nir.write(f"testconv2d+avgpool.nir", nir_graphtest)

In [5]:
nir_graph = nir.read("testconv2d+avgpool.nir")

In [6]:
nir_graph.nodes

{'conv1': Conv2d(input_shape=array([28, 28]), weight=array([[[[-0.02089441, -0.28838855,  0.29146802],
          [ 0.05811119, -0.03130782,  0.10563874],
          [ 0.32223248,  0.20967409,  0.32297143]]],
 
 
        [[[-0.15689926,  0.2150242 , -0.22787003],
          [ 0.22736406, -0.23225805, -0.23206484],
          [ 0.06193602,  0.08859011,  0.02134275]]],
 
 
        [[[ 0.11766748,  0.32074946,  0.32248184],
          [-0.09483679, -0.27267855,  0.06749427],
          [ 0.18732214,  0.32358366, -0.23442106]]],
 
 
        [[[ 0.12948987,  0.02586174,  0.10565313],
          [ 0.05121589,  0.2235953 ,  0.20248985],
          [ 0.04035048, -0.22036462,  0.30777287]]],
 
 
        [[[ 0.01080441, -0.07989316,  0.20943344],
          [ 0.32802436,  0.11138289,  0.08019682],
          [-0.06034279,  0.18786833,  0.3062331 ]]],
 
 
        [[[ 0.2148047 , -0.15108559, -0.09703092],
          [ 0.06732666, -0.25824237, -0.23854768],
          [-0.10475366, -0.11952583, -0.21519837]]]

In [7]:
nir_graph.nodes.keys()

dict_keys(['conv1', 'fc1', 'input', 'lif1', 'lif2', 'output'])

In [8]:
nir_graph.edges

[('lif2', 'output'),
 ('lif1', 'fc1'),
 ('fc1', 'lif2'),
 ('input', 'conv1'),
 ('conv1', 'output')]

In [9]:
nir_graph.nodes["input"]

Input(input_type={'input': array([ 1, 28, 28])})

In [10]:
nir_graph.nodes["conv1"]

Conv2d(input_shape=array([28, 28]), weight=array([[[[-0.02089441, -0.28838855,  0.29146802],
         [ 0.05811119, -0.03130782,  0.10563874],
         [ 0.32223248,  0.20967409,  0.32297143]]],


       [[[-0.15689926,  0.2150242 , -0.22787003],
         [ 0.22736406, -0.23225805, -0.23206484],
         [ 0.06193602,  0.08859011,  0.02134275]]],


       [[[ 0.11766748,  0.32074946,  0.32248184],
         [-0.09483679, -0.27267855,  0.06749427],
         [ 0.18732214,  0.32358366, -0.23442106]]],


       [[[ 0.12948987,  0.02586174,  0.10565313],
         [ 0.05121589,  0.2235953 ,  0.20248985],
         [ 0.04035048, -0.22036462,  0.30777287]]],


       [[[ 0.01080441, -0.07989316,  0.20943344],
         [ 0.32802436,  0.11138289,  0.08019682],
         [-0.06034279,  0.18786833,  0.3062331 ]]],


       [[[ 0.2148047 , -0.15108559, -0.09703092],
         [ 0.06732666, -0.25824237, -0.23854768],
         [-0.10475366, -0.11952583, -0.21519837]]],


       [[[ 0.27725622,  0.2197434

--------------------------------

In [11]:
from torchsummary import summary

class NetWithAvgPool(torch.nn.Module):
    def __init__(self):
        super(NetWithAvgPool, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.lif1 = snn.Leaky(beta=0.9, init_hidden=True)
        self.fc1 = torch.nn.Linear(28*28*16 // 4, 500)  # Adjusted input size after pooling layer
        self.lif2 = snn.Leaky(beta=0.9, init_hidden=True, output=True)

    def forward(self, x):
        x = torch.nn.functional.avg_pool2d(self.conv1(x), kernel_size=2, stride=2)
        x = x.view(-1, 28*28*16 // 4)
        x = self.lif1(x)
        x = self.fc1(x)
        x = self.lif2(x)
        return x

model = NetWithAvgPool()
sample_data = torch.randn(1, 1, 28, 28)  # Assuming input image size is 28x28 and 1 channel
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
             Leaky-2                 [-1, 3136]               0
            Linear-3                  [-1, 500]       1,568,500
             Leaky-4     [[-1, 500], [-1, 500]]               0
Total params: 1,568,660
Trainable params: 1,568,660
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.78
Params size (MB): 5.98
Estimated Total Size (MB): 7.77
----------------------------------------------------------------
